In [1]:
import numpy as np
import matplotlib as plt
from sklearn import linear_model, datasets
import pymysql
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sshtunnel import SSHTunnelForwarder
import random
import functools
import pandas as pd

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='dg',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from dg.searchDB where (crp_cls = 'K' or crp_cls = 'Y')"
    curs.execute(sql)
    rcp_table = curs.fetchall()

server.stop()

In [4]:
len(rcp_table) #2018-08-02 오전 update 기준

1297986

# Rcp Table

In [5]:
rcp_pd = pd.DataFrame.from_dict(rcp_table)

In [6]:
rcp_pd.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12)
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12)
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12)
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12)
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12)


In [12]:
rcp_pd = rcp_pd.drop(columns=['rpt_cat'])

In [13]:
rcp_pd.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_nm
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,사업보고서 (1998.12)
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,사업보고서 (1998.12)
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,사업보고서 (1998.12)
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,사업보고서 (1998.12)
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,사업보고서 (1998.12)


In [14]:
def convertToKorean(x) :
    if x == 'Y': return '유가증권'
    elif x == 'K' : return '코스닥'
    else: return '코넥스'

In [15]:
rcp_pd['crp_cls'] = rcp_pd['crp_cls'].map(convertToKorean)

In [17]:
rcp_pd.tail()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_nm
1297981,080420,코스닥,모다이노칩,모다이노칩,2018-08-02,20180802900089,10:05,코,연결재무제표기준영업(잠정)실적(공정공시)
1297982,072770,코스닥,네오디안테크놀로지,네오디안테크놀로지,2018-08-02,20180802900119,10:45,코,주주총회소집결의
1297983,080160,코스닥,모두투어,모두투어,2018-08-02,20180802900120,10:44,코,기업설명회(IR)개최
1297984,072770,코스닥,네오디안테크놀로지,네오디안테크놀로지,2018-08-02,20180802900122,10:48,코,주식명의개서정지(주주명부폐쇄)
1297985,043590,코스닥,크로바하이텍,크로바하이텍,2018-08-02,20180802900130,11:29,코,유상증자1차발행가액결정


In [18]:
rcp_pd = rcp_pd.rename(columns = {'crp_cd' : '종목코드(고유번호)', 'crp_cls':'법인구분', 'crp_nm' : '종목명(법인명)', 'flr_nm' : '공시 제출인명', 'rcp_dt' : '공시 접수일자', 'rcp_no' : '공시 접수번호', 'rcp_rt' : '공시 접수시간', 'rmk' : '비고', 'rpt_nm' : '공시명'})

In [19]:
rcp_pd.head()

,종목코드(고유번호),법인구분,종목명(법인명),공시 제출인명,공시 접수일자,공시 접수번호,공시 접수시간,비고,공시명
0,023150,유가증권,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,사업보고서 (1998.12)
1,001140,유가증권,국보,국보,1999-03-25,19990325000003,None,정,사업보고서 (1998.12)
2,014160,유가증권,대영포장,대영포장,1999-03-25,19990325000008,None,정연,사업보고서 (1998.12)
3,002310,유가증권,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,사업보고서 (1998.12)
4,015360,유가증권,예스코,예스코,1999-03-25,19990325000016,None,정,사업보고서 (1998.12)


In [20]:
rcp_pd.columns

Index(['종목코드(고유번호)', '법인구분', '종목명(법인명)', '공시 제출인명', '공시 접수일자', '공시 접수번호',
       '공시 접수시간', '비고', '공시명'],
      dtype='object')

In [21]:
cols = ['종목코드(고유번호)', '법인구분', '종목명(법인명)', '공시 접수번호', '공시명','공시 제출인명','공시 접수일자', 
       '공시 접수시간', '비고']
rcp_pd = rcp_pd[cols]

In [23]:
rcp_pd.tail()

,종목코드(고유번호),법인구분,종목명(법인명),공시 접수번호,공시명,공시 제출인명,공시 접수일자,공시 접수시간,비고
1297981,080420,코스닥,모다이노칩,20180802900089,연결재무제표기준영업(잠정)실적(공정공시),모다이노칩,2018-08-02,10:05,코
1297982,072770,코스닥,네오디안테크놀로지,20180802900119,주주총회소집결의,네오디안테크놀로지,2018-08-02,10:45,코
1297983,080160,코스닥,모두투어,20180802900120,기업설명회(IR)개최,모두투어,2018-08-02,10:44,코
1297984,072770,코스닥,네오디안테크놀로지,20180802900122,주식명의개서정지(주주명부폐쇄),네오디안테크놀로지,2018-08-02,10:48,코
1297985,043590,코스닥,크로바하이텍,20180802900130,유상증자1차발행가액결정,크로바하이텍,2018-08-02,11:29,코


In [27]:
sample.tail()

,종목코드(고유번호),법인구분,종목명(법인명),공시 접수번호,공시명,공시 제출인명,공시 접수일자,공시 접수시간,비고
1297981,080420,코스닥,모다이노칩,20180802900089,연결재무제표기준영업(잠정)실적(공정공시),모다이노칩,2018-08-02,10:05,코
1297982,072770,코스닥,네오디안테크놀로지,20180802900119,주주총회소집결의,네오디안테크놀로지,2018-08-02,10:45,코
1297983,080160,코스닥,모두투어,20180802900120,기업설명회(IR)개최,모두투어,2018-08-02,10:44,코
1297984,072770,코스닥,네오디안테크놀로지,20180802900122,주식명의개서정지(주주명부폐쇄),네오디안테크놀로지,2018-08-02,10:48,코
1297985,043590,코스닥,크로바하이텍,20180802900130,유상증자1차발행가액결정,크로바하이텍,2018-08-02,11:29,코


In [24]:
sample = rcp_pd.tail(100)

In [25]:
sample.to_csv('공시_DB_샘플100.txt')

In [28]:
len(rcp_pd)

1297986

In [26]:
rcp_pd.to_csv("공시.txt")

# Close Table

In [29]:
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='dg',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql2 = "select * from dg.stockDB"
    curs.execute(sql2)
    stock_table = curs.fetchall()
    conn.commit()
    conn.close()
    
    
server.stop()

In [30]:
close_pd = pd.DataFrame.from_dict(stock_table)

In [31]:
close_pd.head()

,close,crp_cd,date,datecd,diff,high,low,open,volume
0,36000.0,000020,1996-06-25,19960625000020,0.0,38400.0,36000.0,37700.0,102200.0
1,9300.0,000040,1996-06-25,19960625000040,0.0,9390.0,9290.0,9390.0,1600.0
2,42000.0,000050,1996-06-25,19960625000050,0.0,42000.0,42000.0,42000.0,0.0
3,30500.0,000060,1996-06-25,19960625000060,0.0,31000.0,29700.0,30100.0,68220.0
4,24700.0,000070,1996-06-25,19960625000070,0.0,25600.0,24700.0,25500.0,16960.0


In [32]:
close_pd.tail()

,close,crp_cd,date,datecd,diff,high,low,open,volume
6818445,7100.0,950110,2018-06-29,20180629950110,130.0,7100.0,6850.0,7020.0,13505.0
6818446,4645.0,950130,2018-06-29,20180629950130,5.0,4700.0,4590.0,4645.0,70458.0
6818447,5250.0,950140,2018-06-29,20180629950140,0.0,5350.0,5010.0,5240.0,183640.0
6818448,38400.0,950160,2018-06-29,20180629950160,1050.0,39300.0,36650.0,37600.0,199840.0
6818449,11700.0,950170,2018-06-29,20180629950170,100.0,12150.0,11300.0,11850.0,266087.0


dt = pd.DatetimeIndex(close_pd.date)
close_pd['year'] = dt.year
close_pd['month'] = dt.month
close_pd['day'] = dt.day
close_pd['datecd'] = close_pd['year'].map(str)+close_pd['month'].map(str)+close_pd['day'].map(str)+close_pd['crp_cd']

df['game_date'] = pd.to_datetime(df['game_date'])
df['game_year'] = df['game_date'].dt.year
df['game_month'] = df['game_date'].dt.month
df['game_day'] = df['game_date'].dt.dayofweek

In [37]:
close_pd = close_pd.rename(columns = {'crp_cd' : '종목코드(고유번호)', 'date':'날짜', 'diff' : '전일비', 'high' : '고가', 'low' : '저가', 'open' : '시가', 'volume' : '거래량', 'close' : '종가'})

In [38]:
cols = ['종목코드(고유번호)', '날짜', '고가', '저가', '시가', '종가', '전일비', '거래량']
close_pd = close_pd[cols]

In [41]:
samp = close_pd.tail(100)

In [ ]:
samp.head()

In [ ]:
samp

In [43]:
samp.to_csv("주가_DB_샘플100.txt")

In [44]:
close_pd.to_csv("주가.txt")

# to_csv

In [205]:
conn = pymysql.connect(host='localhost', user='root', password='qlqtmajrwk',
                       db='yejinyDB', charset='utf8')
 
# Connection 으로부터 Dictoionary Cursor 생성
curs = conn.cursor(pymysql.cursors.DictCursor)

In [209]:
sql = "SELECT * FROM yejinyDB.Rcp where now_st != -1;"
curs.execute(sql)
zziny_rcp_table = curs.fetchall()

conn.commit()
conn.close()
    
server.stop()

InterfaceError: (0, '')

In [218]:
zziny_rcp = pd.DataFrame.from_dict(zziny_rcp_table)

In [224]:
zziny_rcp.to_csv('실시간공시실시간주가.csv', encoding='utf-8')

In [235]:
merged_pd

,close,crp_cd_x,date_x,datecd,diff,high,low,open,volume,year_x,...,now_volume,date_y,rcp_no,rmk,rpt_cat,rpt_link,rpt_nm,year_y,month_y,day_y


In [234]:
merged_pd.to_csv('공시와회사merge.csv', encoding='utf-8')

In [241]:
close_pd = close_pd['year'=2017]

SyntaxError: invalid syntax (<ipython-input-241-b6b09a37e8ee>, line 1)